# My Dinning Spending Habits
The purpose of this notebook is to examine my dinning spending habits during my years in college. I will be using data I receieving from Rit Dining from 08/2021 to 1/2022.

## Imports

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Step One: Gathering the Data

In [ ]:
august_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-08-01_to_2021-08-31.csv')
september_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-09-01_to_2021-09-30.csv')
october_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-10-01_to_2021-10-31.csv')
november_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-11-01_to_2021-11-30.csv')
december_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-12-01_to_2021-12-14.csv')
january_df = pd.read_csv('sample-data/Dining Dollars (Meal Plan)_statement_2021-12-16_to_2022-01-31.csv')


In [ ]:
august_df.head(3)

### Storing the DataFrame into lists catagorized by their semester

In [ ]:
fall_semester = [august_df, september_df, october_df, november_df, december_df]
spring_semester = [january_df]
all_semesters = [fall_semester, spring_semester]

## Step Two: Cleaning the Data

### Helper Functions to Clean Up Unneeded Info in the Description Column

In [ ]:
def remove_brackets(row):
    return row.split(' [')[0]

def remove_num_at_end(row):
    if row[len(row)-1].isnumeric(): 
        return row[0:len(row)-2]
    else:
        return row

def remove_bad_suffixes(row):
    row = row.removesuffix(' OnDemand')
    row = row.removesuffix(' - Tablet')
    return row

def clean_up_description(row):
    row = remove_brackets(row)
    row = remove_num_at_end(row)
    row = remove_bad_suffixes(row)
    return row

### Helper Function for Creating New Time Columns

In [ ]:
day_of_week_map = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

def create_time_columns(t):
    return pd.Series({'Year': t.year, 'Month': t.month, 'Day Of Week': t.day_of_week, 'Hour': t.hour})

### Cleaning Up DataFrames and Combining Them Based on Semester

In [ ]:
def clean_semester_data(semester):
    for i in range(0, len(semester)):
        semester[i].sort_values(by='Date', inplace=True)
        semester[i]['Description'] = semester[i]['Description'].apply(clean_up_description)
        semester[i]['Date'] = pd.to_datetime(semester[i]['Date'])
        semester[i] = pd.concat([semester[i], semester[i]['Date'].apply(create_time_columns)], axis=1)
        semester[i]['Day Of Week'] = semester[i]['Day Of Week'].map(day_of_week_map)
        semester[i] = semester[i][['Description', 'Amount', 'Balance', 'Date', 'Year', 'Month', 'Day Of Week', 'Hour']]
    return semester

def aggregate_semester_data(semester):
    semester = clean_semester_data(semester)

    semester_df = semester[0]
    for index in range(1, len(semester)):
        semester_df = pd.concat([semester_df, semester[index]])

    # semester_df.sort_values(by='Date', inplace=True)
    semester_df = semester_df.reset_index()
    semester_df.drop('index', axis=1, inplace=True)

    return semester_df

In [ ]:
fall_semester_df = aggregate_semester_data(fall_semester)
spring_semester_df = aggregate_semester_data(spring_semester)

## Step Three: Modeling and Analysing the Data

### The Fall Semester

In [ ]:
fall_semester[0].head()

In [ ]:
fall_semester_df.info()

In [ ]:
fall_semester_df

#### How Much Did I Spend This Semester

In [ ]:
fall_semester_df['Amount'].sum()

In [ ]:
fall_semester_df['Balance'].iloc[299]

### The Spring Semester

In [ ]:
spring_semester_df.info()

In [ ]:
spring_semester_df.head()